In [0]:
import json
from glob import glob
from functions.sanity import validate_settings, initialize_empty_tables
from functions.job import save_job_configuration
from functions.utility import apply_job_type

try:
    save_job_config = dbutils.widgets.get('save_job_config')
except Exception:
    save_job_config = 'false'
save_job_config = str(save_job_config).lower()

if save_job_config == 'true':
    try:
        catalog = dbutils.widgets.get('catalog')
    except Exception as e:
        raise ValueError("The 'catalog' widget is required when saving the job configuration. Please set it with the Jobs & Workflows web interface before running this job.") from e

    try:
        bronze_schema = dbutils.widgets.get('bronze_schema')
    except Exception as e:
        raise ValueError("The 'bronze_schema' widget is required when saving the job configuration. Please set it with the Jobs & Workflows web interface before running this job.") from e

# Load anything in layer_*_<color>/*.json
job_settings = {}
for color in [ "bronze", "silver", "gold" ]:
    paths = glob(f'./layer_*_{color}/*.json')
    job_settings[color] = [
        {"table": path.split("/")[-1].split(".")[0]}
        for path in paths]
for key, value in job_settings.items():
    dbutils.jobs.taskValues.set(key=key, value=value)

# Load anything in layer_*/*.json if it has a key "dst_table_name"
history_settings = []
for p in glob('./layer_*/*.json'):
    with open(p) as f:
        settings = json.load(f)
        settings = apply_job_type(settings)
        if str(settings.get("build_history", "false")).lower() == "true":
            entry = {"full_table_name": settings["dst_table_name"]}
            if "history_schema" in settings:
                entry["history_schema"] = settings["history_schema"]
            history_settings.append(entry)
dbutils.jobs.taskValues.set(key="history_settings", value=history_settings)

# Sanity check
validate_settings(dbutils)
initialize_empty_tables(spark)

if save_job_config == 'true':
    # Save job configuration
    save_job_configuration(dbutils, f"/Volumes/{catalog}/{bronze_schema}/utility/jobs")